# Importing Libs

In [260]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import re

from numpy import array
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Embedding
from tensorflow.keras.utils import to_categorical

# Loading Dataset

In [177]:
dataset = pd.read_csv('dataset.tsv', sep='\t',header=0 ,encoding="utf-8",
                      names=['rating', 'review_id', 'user_id', 'book_id', 'review'])

In [178]:
dataset.head()

,rating,review_id,user_id,book_id,review
0,4,39428407,1775679,3554772,من أمتع ما قرأت من روايات بلا شك. وحول الشك ت...
1,4,32159373,1304410,3554772,رواية تتخذ من التاريخ ،جوًا لها اختار المؤلف ...
2,1,442326656,11333112,3554772,إني أقدّر هذه الرواية كثيرا، لسبب مختلف عن أس...
3,5,46492258,580165,3554772,الكاهن الذي أطلق على نفسه اسم هيبا تيمنا بالع...
4,5,25550893,1252226,3554772,"""عزازيل"" هو اسم رواية يوسف زيدان الثانية و ال..."


In [189]:
dataset = dataset.loc[:, ['review', 'rating']]

In [342]:
dataset['review'][2]

'اني اقدر هذه الروايه كثيرا  لسبب مختلف عن اسباب الاخرين  الا وهو انها علمتني درسا قيما  حتي وان كانت العبره قد اتت بعد فوات الاوان  وذلك عهدنا بدروس الحياه  علمتني الروايه ان الصديق الذي يشجعك علي قراءه مثيلاتها دون اي اشاره الي المقاطع الاباحيه التي هي فيها  هو حتما صديق غير جدير بالثقه   قد تقولون عن تعميمي انه خاطئ  واقول لكم  فلتاتوا وتحاسبوني ان خجل صديقكم ذاك من قله ادبه  او ان ظل علي احترامه لاخلاقكم  وهما امران مستبعدان  جدا   توقفت عن قراءه الروايه لما بدا مستواها الاخلاقي يسقط الي الحضيض  وقد صدمت به صدمه قويه  فقلت لنفسي  فلالتمس بعض المواساه في ماساتي بقراءه شيء من اراء القراء في هذا الموقع  لا شك انهم مفجوعون مثلي  ثم كانت الصدمه الثانيه    اغلب التقييمات نجوم خمس او اربع علي الاقل  وتتلو تلك الصدمه مباشره صدمه اخيره هي في الاراء بحد ذاتها  اذ تري معظم كاتبيها بين مادح للاسلوب المذهل  ومطبل لعبقريه يوسف زيدان  ومصفق لروعه الاحداث  وبين جامع لها كلها دفعه واحده   وان تكرم احدهم فهو يذيل تعليقه بقول من مثل   لكني كرهت الاباحيه التي فيها   والسلام     خاتمه مدهشه  تكلمت فاوج

# Some preprocessing 

In [206]:
def normalizeArabic(text):
    text = text.strip()
    text = re.sub(r"[إأٱآا]", "ا", text)
    text = re.sub(r"ى", "ي", text)
    text = re.sub(r"ة","ه", text)
    text = re.sub(r"[0-9]|[!؟،؛,-_]", " ", text)
    text = re.sub(r'"', " ", text)
    noise = re.compile(""" ّ    | # Tashdid
                             َ    | # Fatha
                             ً    | # Tanwin Fath
                             ُ    | # Damma
                             ٌ    | # Tanwin Damm
                             ِ    | # Kasra
                             ٍ    | # Tanwin Kasr
                             ْ    | # Sukun
                             ـ     # Tatwil/Kashida
                         """, re.VERBOSE)
    
    text = re.sub(noise, '', text)
    return ''.join(text)

## Applying preprocessing

In [207]:
dataset['review'] = dataset['review'].apply(normalizeArabic)

# Squeezing reviews

In [209]:
reviews = [review for review in dataset['review']]

In [210]:
reviews

['من امتع ما قرات من روايات بلا شك  وحول الشك تدندن (عزازيل) بلا هواده  احمد الديب',
 'روايه تتخذ من التاريخ  جوا لها اختار المؤلف فتره تاريخيه ندر من يتناولها روائيا  مكتوبه بدقه واتقان وجمال من اروع ما يمكن ان تقرا من الروايات التاريخيه  تركز علي الانسان صانع المعني ومدمره',
 'اني اقدر هذه الروايه كثيرا  لسبب مختلف عن اسباب الاخرين  الا وهو انها علمتني درسا قيما  حتي وان كانت العبره قد اتت بعد فوات الاوان  وذلك عهدنا بدروس الحياه  علمتني الروايه ان الصديق الذي يشجعك علي قراءه مثيلاتها دون اي اشاره الي المقاطع الاباحيه التي هي فيها  هو حتما صديق غير جدير بالثقه   قد تقولون عن تعميمي انه خاطئ  واقول لكم  فلتاتوا وتحاسبوني ان خجل صديقكم ذاك من قله ادبه  او ان ظل علي احترامه لاخلاقكم  وهما امران مستبعدان  جدا   توقفت عن قراءه الروايه لما بدا مستواها الاخلاقي يسقط الي الحضيض  وقد صدمت به صدمه قويه  فقلت لنفسي  فلالتمس بعض المواساه في ماساتي بقراءه شيء من اراء القراء في هذا الموقع  لا شك انهم مفجوعون مثلي  ثم كانت الصدمه الثانيه    اغلب التقييمات نجوم خمس او اربع علي الاقل  وتتلو تلك الصدم

# Splitting

In [371]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(reviews, dataset['rating']
                                                    , test_size=0.1, random_state=0)

# Making ratings from 0 to 4 instead 1 to 5

In [372]:
dataset['rating'] = dataset['rating'] - 1

In [373]:
y_train = to_categorical(y_train, 5)
y_test = to_categorical(y_test, 5)

## This code is to know the total number of words in the whole reviews ,so that I know the vocab size

In [317]:
length = 0
for review in reviews:
    review = review.split()
    length += len(review)

In [318]:
length

191626

In [374]:
vocab_size = 200000
encoded_docs_train = [one_hot(review, vocab_size) for review in x_train]
encoded_docs_test = [one_hot(review, vocab_size) for review in x_test]

## This code is to know the maximum review in length ,so that I know the padding nomber

In [375]:
max([len(sublist.split()) for sublist in reviews]) 

3086

In [376]:
max_length = 3086   # There is no wrong if putting a number bigger than 3086 but this will be consuming in ram
padded_docs_train = pad_sequences(encoded_docs_train, maxlen=max_length, padding='post')
padded_docs_test = pad_sequences(encoded_docs_test, maxlen=max_length, padding='post') 

In [367]:
# from imblearn.over_sampling import SMOTE   #'Synthetic Minority Oversampling TEchnique'
# oversample = SMOTE()
# padded_docs_train, y_train = oversample.fit_resample(padded_docs_train, y_train)
# padded_docs_test, y_test = oversample.fit_resample(padded_docs_test, y_test)

# Feature extraction Using Embedding Layer and building the model

In [377]:
model = Sequential()
model.add(Embedding(vocab_size, 32, input_length=max_length)) 
# Before embedding each sentence was vector of 3086 dimension, after it each sentence was vector of 32 dim
model.add(Flatten())
# model.add(Dense(50, activation='relu'))
model.add(Dense(5, activation='softmax'))
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [378]:
model.summary()

Model: "sequential_19"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_19 (Embedding)     (None, 3086, 32)          6400000   
_________________________________________________________________
flatten_19 (Flatten)         (None, 98752)             0         
_________________________________________________________________
dense_20 (Dense)             (None, 5)                 493765    
Total params: 6,893,765
Trainable params: 6,893,765
Non-trainable params: 0
_________________________________________________________________


# Training the model

In [379]:
model.fit(padded_docs_train, y_train, epochs=15,validation_data=(padded_docs_test, y_test) ,verbose=1)

Epoch 1/15
88/88 [==============================] - 8s 89ms/step - loss: 1.2082 - accuracy: 0.5768 - val_loss: 1.0496 - val_accuracy: 0.6410
Epoch 2/15
88/88 [==============================] - 7s 81ms/step - loss: 1.0276 - accuracy: 0.6086 - val_loss: 1.0525 - val_accuracy: 0.6218
Epoch 3/15
88/88 [==============================] - 7s 83ms/step - loss: 0.7665 - accuracy: 0.7354 - val_loss: 0.9972 - val_accuracy: 0.6154
Epoch 4/15
88/88 [==============================] - 7s 81ms/step - loss: 0.4997 - accuracy: 0.8418 - val_loss: 0.9996 - val_accuracy: 0.6122
Epoch 5/15
88/88 [==============================] - 7s 81ms/step - loss: 0.3329 - accuracy: 0.9043 - val_loss: 1.0196 - val_accuracy: 0.5962
Epoch 6/15
88/88 [==============================] - 7s 82ms/step - loss: 0.2348 - accuracy: 0.9432 - val_loss: 1.0431 - val_accuracy: 0.5994
Epoch 7/15
88/88 [==============================] - 7s 83ms/step - loss: 0.1741 - accuracy: 0.9618 - val_loss: 1.0541 - val_accuracy: 0.5865
Epoch 8/15
88

# The Generalization of the model on the test data is bad because of unbalancing of the data ,and trying SMOTE here is meaningless because the representation of the sentence is random because one_hot function give each word in each sentence a random number.

In [381]:
model.save('Arabic Book Review with random-initialized Embedding layer.h5')

# Test Case

In [331]:
inpt = 'من امتع ما قرات من روايات بلا شك  وحول الشك تدندن (عزازيل) بلا هواده  احمد الديب'
encoded_inpt = [one_hot(inpt, vocab_size)]
padded_inpt = pad_sequences(encoded_inpt, maxlen=max_length, padding='post')
np.argmax(model.predict(padded_inpt))

3

# 3 is the correct solution actually

In [345]:
# Another test case nt found in the reviews
inpt = 'يا خساره الوقت والمال'
inpt = normalizeArabic(inpt)
encoded_inpt = [one_hot(inpt, vocab_size)]
padded_inpt = pad_sequences(encoded_inpt, maxlen=max_length, padding='post')
model.predict(padded_inpt)

array([[0.00571362, 0.00518488, 0.02255757, 0.19787161, 0.76867235]],
      dtype=float32)

# As I said previously the Generalization of the model on the test data is bad because of the unbalancing in the data .